<a href="https://colab.research.google.com/github/ahmedsaadalvi/Dataset-Preprocessing-Python/blob/main/Dataset_Preprocessing_with_TensorFlow_Transform_(TFT).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow-transform==<desired_version>
!pip install apache-beam

import tempfile
import pandas as pd
import tensorflow as tf
import tensorflow_transform as tft
import tensorflow_transform.beam.impl as tft_beam

from __future__ import print_function
from tensorflow_transform.tf_metadata import dataset_schema, dataset_metadata

dataset = pd.read_csv("pollution_small.csv")
dataset.head()

features = dataset.drop("Date", axis=1)
features.head()

dict_features = list(features.to_dict("index").values())
dict_features[:2]

data_metadata = dataset_metadata.DatasetMetadata(
    dataset_schema.from_feature_spec({
        "no2": tf.io.FixedLenFeature([], tf.float32),
        "so2": tf.io.FixedLenFeature([], tf.float32),
        "pm10": tf.io.FixedLenFeature([], tf.float32),
        "soot": tf.io.FixedLenFeature([], tf.float32),
    })
)

def preprocessing_fn(inputs):

    no2 = inputs['no2']
    pm10 = inputs['pm10']
    so2 = inputs['so2']
    soot = inputs['soot']

    no2_normalized = no2 - tft.mean(no2)
    so2_normalized = so2 - tft.mean(so2)

    pm10_normalized = tft.scale_to_0_1(pm10)
    soot_normalized = tft.scale_by_min_max(soot)

    return {
        "no2_normalized":no2_normalized,
        "so2_normalized":so2_normalized,
        "pm10_normalized":pm10_normalized,
        "soot_normalized":soot_normalized
    }

def data_transform():
    with tft_beam.Context(temp_dir=tempfile.mkdtemp()):
        transformed_dataset, transform_fn = ((dict_features, data_metadata) | tft_beam.AnalyzeAndTransformDataset(preprocessing_fn))

    transformed_data, transformed_metadata = transformed_dataset

    for i in range(len(transformed_data)):
        print("Raw: ", dict_features[i])
        print("Transformed:", transformed_data[i])

data_transform()

/bin/bash: -c: line 1: syntax error near unexpected token `newline'
/bin/bash: -c: line 1: `pip install tensorflow-transform==<desired_version>'


ImportError: cannot import name 'dataset_schema' from 'tensorflow_transform.tf_metadata' (/usr/local/lib/python3.10/dist-packages/tensorflow_transform/tf_metadata/__init__.py)